# TP2.2 : Stratégies d’investissement (bis)
Notebook fait par Alexandre Boistard, Ethan Trentin.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import math
import seaborn as sns
from tqdm import tqdm

In [5]:
# question 0 : import des données
df = pd.read_parquet("sp500_20161229.parquet")
print(df.head())

        index         A        AA      AABA       ABC       ABT      ADBE  \
0  2006-01-03  0.006308  0.011160  0.044155  0.020773  0.003043  0.049784   
1  2006-01-04  0.002687  0.005686  0.001467 -0.006626  0.001517 -0.009794   
2  2006-01-05  0.026198  0.007981  0.013669 -0.012387  0.010856 -0.009110   
3  2006-01-06  0.005222 -0.003299  0.040453 -0.011095  0.021229  0.024429   
4  2006-01-09 -0.002886  0.011255  0.004860  0.000000  0.037173 -0.015641   

        ADI       ADP      ADSK  ...  WU        WY       XEL        XL  \
0  0.023139  0.009368 -0.004657  ... NaN  0.004522  0.005959  0.012615   
1  0.009264  0.003238 -0.005951  ... NaN  0.024160  0.004847  0.020519   
2  0.031857 -0.008391 -0.014722  ... NaN -0.008791 -0.000536 -0.001867   
3  0.003925  0.008679  0.057798  ... NaN  0.001478  0.003753  0.001583   
4  0.014073 -0.002581 -0.014676  ... NaN -0.002362 -0.002671  0.006752   

       XLNX       XOM       XRX       YUM       ZBH      ZION  
0  0.022213  0.040947  0.017

In [2]:
# Question 1. Nous allons d'abord implémenter des stratégies du tp 2.1
def ema_crossover_strategy(df, tickers, short_window, long_window):
    signals = pd.DataFrame(index=df.index)
    for stock in tickers:
        # Compute the short and long EMAs. Change half_life to span if desired.
        signals[stock + " Short EMA"] = df[stock].ewm(halflife =short_window).mean()
        signals[stock + " Long EMA"] = df[stock].ewm(halflife =long_window).mean()
        # Generate signals: 1 if short EMA > long EMA, else -1.
        signals[stock + " Signal"] = np.where(
            signals[stock + " Short EMA"] > signals[stock + " Long EMA"],
            1,
            -1
        )
    return signals

In [3]:
def mean_revert_strategy(df, tickers, time_window, k):
    signals = pd.DataFrame(index=df.index)
    for stock in tickers:
        # Compute the MA.
        signals[stock + " MA"] = df[stock].rolling(window=time_window).mean()
        # Generate signals: -1 if > k*simga, 0 between, else 1.
        signals[stock + " Signal"] = np.where(
            df[stock] > signals[stock + " MA"] + k * df[stock].rolling(window=time_window).std(),
            -1,
            np.where(
                df[stock] < signals[stock + " MA"] - k * df[stock].rolling(window=time_window).std(),
                1,
                0
            )
        )
    return signals

In [ ]:
# calculer la performance des stratégies
def compute_performance(df,strategy,metric='avg_log_return'):
    df_ = df.copy()
    for ticker in df_.colums().remove('index'):
        signals = strategy(df_,[ticker],*args)